In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Load data from URL

In [13]:
data_url = 'https://raw.githubusercontent.com/LAfishableman/Homework-5/refs/heads/main/u.data'
ratings = pd.read_csv(data_url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

Create the utility matrix

In [14]:
utility_matrix = ratings.pivot(index='user_id', columns='item_id', values='rating')

Centralize ratings

In [15]:
user_means = utility_matrix.mean(axis=1)
centered_utility_matrix = utility_matrix.sub(user_means, axis=0)
centered_utility_matrix_filled = centered_utility_matrix.fillna(0)

Compute cosine similarity

In [16]:
cosine_sim = cosine_similarity(centered_utility_matrix_filled)
cosine_sim_df = pd.DataFrame(cosine_sim, index=utility_matrix.index, columns=utility_matrix.index)

Get top 10 similar users to user 1

In [17]:
user_1_sim = cosine_sim_df.loc[1].sort_values(ascending=False)[1:11]

Predict rating for movie 508

In [18]:
mu_1 = user_means.loc[1]
similar_users = user_1_sim.index
weighted_sum = 0
sim_sum = 0

for user in similar_users:
    if not pd.isna(utility_matrix.loc[user, 508]):
        sim_score = user_1_sim.loc[user]
        rating = utility_matrix.loc[user, 508]
        mu_u = user_means.loc[user]
        weighted_sum += sim_score * (rating - mu_u)
        sim_sum += sim_score

predicted_rating = mu_1 + (weighted_sum / sim_sum) if sim_sum > 0 else mu_1

Print results

In [19]:
print("Top 10 users similar to user 1:")
print(user_1_sim)
print(f"\nUser 1 predicted rating for movie 508: {predicted_rating:.2f}")

Top 10 users similar to user 1:
user_id
773    0.204792
868    0.202321
592    0.196592
880    0.195801
429    0.190661
276    0.187476
916    0.186358
222    0.182415
457    0.182253
8      0.180891
Name: 1, dtype: float64

User 1 predicted rating for movie 508: 4.39
